<a href="https://colab.research.google.com/github/Sandhya-G/Aggression-Detection/blob/master/notebook/Corelation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import time
import numpy as np 
import pandas as pd 
from tqdm import tqdm
import math
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf
import keras

from keras.layers import Dense, Input, Embedding, Dropout, Activation, GRU, Conv1D, concatenate
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D, BatchNormalization
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.callbacks import LambdaCallback

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import *
from keras.models import *
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K






Using TensorFlow backend.


In [6]:
custom_objects={'Attention': Attention}

In [5]:
#custom attention layer refer the second figure and keras documentation ‘Writing your own Keras layer'. 
class Attention(Layer):
    #initialise
    def __init__(self, step_dim=120,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        # Builds all weights
        # W = Weight matrix, b = bias vector, u = context vector
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        #calculations
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim




In [80]:
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

## split to train and val

train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=2048,stratify=train_df["class"])

## some config values 
embed_size = 100 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a question to use

## fill up the missing values
train_X = train_df["tweet"].fillna("_na_").values
val_X = val_df["tweet"].fillna("_na_").values
test_X = test_df["tweet"].fillna("_na_").values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen,padding='post',truncating='post')
val_X = pad_sequences(val_X, maxlen=maxlen,padding='post',truncating='post')
test_X = pad_sequences(test_X, maxlen=maxlen,padding='post',truncating='post')

## Get the target values
train_y = train_df['class'].values
val_y = val_df['class'].values
test_y = test_df['class'].values


        



In [59]:
model = load_model('/content/attention.h5',custom_objects=custom_objects)

In [73]:
model = load_model('/content/cnn_text.h5')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [75]:

train_df = pd.read_csv("https://raw.githubusercontent.com/Sandhya-G/Aggression-Detection/master/dataset/GLOVE_TRAIN%23")
test_df = pd.read_csv("https://raw.githubusercontent.com/Sandhya-G/Aggression-Detection/master/dataset/GLOVE_TEST%23")

In [81]:
p = model.predict([test_X])
pd.DataFrame(p).to_csv('/content/cnn_text.csv')


In [ ]:
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

## split to train and val
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=2048,stratify=train_df["class"])

## some config values 
embed_size = 100 # how big is each word vector
max_features = 30000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 120 # max number of words in a question to use

## fill up the missing values
train_X = train_df["tweet"].fillna("_na_").values
val_X = val_df["tweet"].fillna("_na_").values
test_X = test_df["tweet"].fillna("_na_").values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features,filters="")
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen,padding='post',truncating='post')
val_X = pad_sequences(val_X, maxlen=maxlen,padding='post',truncating='post')
test_X = pad_sequences(test_X, maxlen=maxlen,padding='post',truncating='post')

## Get the target values
train_y = train_df['class'].values
val_y = val_df['class'].values


In [65]:
import pandas as pd
import sys

first_file = '/content/attention.csv'
second_file = '/content/gru.csv'

def corr(first_file, second_file):
  first_df = pd.read_csv(first_file,index_col=0)
  second_df = pd.read_csv(second_file,index_col=0)
  # assuming first column is `prediction_id` and second column is `prediction`
  prediction = first_df.columns[0]
  # correlation
  print("Finding correlation between: {} and {}".format(first_file,second_file))
  print("Column to be measured: {}".format(prediction))
  #print(" score: {}".format(first_df[prediction]))
  #print(" score1: {}".format(second_df[prediction])
  
  print("Pearson's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='pearson')))
  print("Kendall's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='kendall')))
  print("Spearman's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='spearman')))

corr(first_file, second_file)

Finding correlation between: /content/attention.csv and /content/gru.csv
Column to be measured: 0
Pearson's correlation score: 0.12292177257573225
Kendall's correlation score: 0.15997860780882062
Spearman's correlation score: 0.2364503153857585


In [68]:
import pandas as pd
import sys

first_file = '/content/attention.csv'
second_file = '/content/cnn.csv'

def corr(first_file, second_file):
  first_df = pd.read_csv(first_file,index_col=0)
  second_df = pd.read_csv(second_file,index_col=0)
  # assuming first column is `prediction_id` and second column is `prediction`
  prediction = first_df.columns[0]
  # correlation
  print("Finding correlation between: {} and {}".format(first_file,second_file))
  print("Column to be measured: {}".format(prediction))
  print("Pearson's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='pearson')))
  print("Kendall's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='kendall')))
  print("Spearman's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='spearman')))

corr(first_file, second_file)

Finding correlation between: /content/attention.csv and /content/cnn.csv
Column to be measured: 0
Pearson's correlation score: 0.11565194088081888
Kendall's correlation score: 0.1361165204597667
Spearman's correlation score: 0.20309125425478336


In [69]:
import pandas as pd
import sys

first_file = '/content/gru.csv'
second_file = '/content/cnn.csv'

def corr(first_file, second_file):
  first_df = pd.read_csv(first_file,index_col=0)
  second_df = pd.read_csv(second_file,index_col=0)
  # assuming first column is `prediction_id` and second column is `prediction`
  prediction = first_df.columns[0]
  # correlation
  print("Finding correlation between: {} and {}".format(first_file,second_file))
  print("Column to be measured: {}".format(prediction))
  print("Pearson's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='pearson')))
  print("Kendall's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='kendall')))
  print("Spearman's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='spearman')))

corr(first_file, second_file)

Finding correlation between: /content/gru.csv and /content/cnn.csv
Column to be measured: 0
Pearson's correlation score: 0.9183782615552456
Kendall's correlation score: 0.7284301225797093
Spearman's correlation score: 0.9093209091303897


In [37]:
import pandas as pd
import sys

first_file = '/content/attention.csv'
second_file = '/content/cnn1.csv'

def corr(first_file, second_file):
  first_df = pd.read_csv(first_file,index_col=0)
  second_df = pd.read_csv(second_file,index_col=0)
  # assuming first column is `prediction_id` and second column is `prediction`
  prediction = first_df.columns[0]
  # correlation
  print("Finding correlation between: {} and {}".format(first_file,second_file))
  print("Column to be measured: {}".format(prediction))
  print("Pearson's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='pearson')))
  print("Kendall's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='kendall')))
  print("Spearman's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='spearman')))

corr(first_file, second_file)

Finding correlation between: /content/attention.csv and /content/cnn1.csv
Column to be measured: Unnamed: 1
Pearson's correlation score: 0.8968351277718838
Kendall's correlation score: 0.9191193905525553
Spearman's correlation score: 0.9191254428230831


In [ ]:
import pandas as pd
import sys

first_file = '/content/gru.csv'
second_file = '/content/cnn1.csv'

def corr(first_file, second_file):
  first_df = pd.read_csv(first_file,index_col=0)
  second_df = pd.read_csv(second_file,index_col=0)
  # assuming first column is `prediction_id` and second column is `prediction`
  prediction = first_df.columns[0]
  # correlation
  print("Finding correlation between: {} and {}".format(first_file,second_file))
  print("Column to be measured: {}".format(prediction))
  print("Pearson's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='pearson')))
  print("Kendall's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='kendall')))
  print("Spearman's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='spearman')))

corr(first_file, second_file)

Finding correlation between: /content/gru.csv and /content/cnn1.csv
Column to be measured: Unnamed: 1
Pearson's correlation score: 0.902049021719535


In [83]:
import pandas as pd
import sys

first_file = '/content/attention.csv'
second_file = '/content/cnn_text.csv'

def corr(first_file, second_file):
  first_df = pd.read_csv(first_file,index_col=0)
  second_df = pd.read_csv(second_file,index_col=0)
  # assuming first column is `prediction_id` and second column is `prediction`
  prediction = first_df.columns[0]
  # correlation
  print("Finding correlation between: {} and {}".format(first_file,second_file))
  print("Column to be measured: {}".format(prediction))
  #print(" score: {}".format(first_df[prediction]))
  #print(" score1: {}".format(second_df[prediction])
  
  print("Pearson's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='pearson')))
  print("Kendall's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='kendall')))
  print("Spearman's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='spearman')))

corr(first_file, second_file)

Finding correlation between: /content/attention.csv and /content/cnn_text.csv
Column to be measured: 0
Pearson's correlation score: -0.04327056651050293
Kendall's correlation score: 0.11626124799034313
Spearman's correlation score: 0.17753628368694008


In [84]:
import pandas as pd
import sys

first_file = '/content/gru.csv'
second_file = '/content/cnn_text.csv'

def corr(first_file, second_file):
  first_df = pd.read_csv(first_file,index_col=0)
  second_df = pd.read_csv(second_file,index_col=0)
  # assuming first column is `prediction_id` and second column is `prediction`
  prediction = first_df.columns[0]
  # correlation
  print("Finding correlation between: {} and {}".format(first_file,second_file))
  print("Column to be measured: {}".format(prediction))
  #print(" score: {}".format(first_df[prediction]))
  #print(" score1: {}".format(second_df[prediction])
  
  print("Pearson's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='pearson')))
  print("Kendall's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='kendall')))
  print("Spearman's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='spearman')))

corr(first_file, second_file)

Finding correlation between: /content/gru.csv and /content/cnn_text.csv
Column to be measured: 0
Pearson's correlation score: 0.10524120585500138
Kendall's correlation score: 0.2891194596411396
Spearman's correlation score: 0.4386581267451317


In [86]:
import pandas as pd
import sys

first_file = '/content/cnn.csv'
second_file = '/content/cnn_text.csv'

def corr(first_file, second_file):
  first_df = pd.read_csv(first_file,index_col=0)
  second_df = pd.read_csv(second_file,index_col=0)
  # assuming first column is `prediction_id` and second column is `prediction`
  prediction = first_df.columns[0]
  # correlation
  print("Finding correlation between: {} and {}".format(first_file,second_file))
  print("Column to be measured: {}".format(prediction))
  #print(" score: {}".format(first_df[prediction]))
  #print(" score1: {}".format(second_df[prediction])
  
  print("Pearson's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='pearson')))
  print("Kendall's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='kendall')))
  print("Spearman's correlation score: {}".format(first_df[prediction].corr(second_df[prediction],method='spearman')))

corr(first_file, second_file)

Finding correlation between: /content/cnn.csv and /content/cnn_text.csv
Column to be measured: 0
Pearson's correlation score: 0.10438494501230992
Kendall's correlation score: 0.2883153422521823
Spearman's correlation score: 0.4296728248524211


In [11]:
def predict_result(text):
  val = tokenizer.texts_to_sequences([text])
  val = pad_sequences(val, maxlen=maxlen,padding='post',truncating='post')
  return model.predict(val)[0]

In [12]:
predict_result("That is fucking good")

array([0.17055526], dtype=float32)

In [13]:
predict_result("black lives matter")

array([0.11282898], dtype=float32)

In [19]:
predict_result("you suck")

array([0.61150247], dtype=float32)